In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Load your Task 1 dataset
df = pd.read_csv('/content/Nat_Gas.csv')
df.columns = ["Date", "Price"]
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")


/tmp/ipython-input-3159052536.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


# Rebuild get_price() from Task 1

In [3]:
# Seasonality factors
seasonality = {
    1:1.10, 2:1.12, 3:1.05, 4:0.98,
    5:0.90, 6:0.88, 7:0.92, 8:0.95,
    9:1.00, 10:1.05, 11:1.08, 12:1.12
}

df["Month"] = df["Date"].dt.month
df["SeasonalPrice"] = df["Price"] * df["Month"].map(seasonality)

# Daily series
df_daily = df[["Date","SeasonalPrice"]].set_index("Date").resample("D").mean()
df_daily["SeasonalPrice"] = df_daily["SeasonalPrice"].interpolate(method="cubic")

# Extrapolation for 1 year
last_date = df_daily.index.max()
future_dates = pd.date_range(last_date + timedelta(days=1), periods=365)

last_30 = df_daily["SeasonalPrice"][-30:]
daily_slope = (last_30.iloc[-1] - last_30.iloc[0]) / 29

future_prices = last_30.iloc[-1] + daily_slope * np.arange(1, 366)

df_future = pd.DataFrame({"SeasonalPrice": future_prices}, index=future_dates)
df_extended = pd.concat([df_daily, df_future])


# Final get_price Function

In [4]:
def get_price(date_str):
    date = pd.to_datetime(date_str)

    if date < df_extended.index.min():
        return float(df_extended.iloc[0]["SeasonalPrice"])

    if date in df_extended.index:
        return float(df_extended.loc[date]["SeasonalPrice"])

    if df_extended.index.min() <= date <= df_extended.index.max():
        return float(np.interp(
            date.timestamp(),
            df_extended.index.astype(np.int64) // 10**9,
            df_extended["SeasonalPrice"]
        ))

    extra_days = (date - df_extended.index.max()).days
    return float(df_extended.iloc[-1]["SeasonalPrice"] + daily_slope * extra_days)


# Storage Contract Pricing Function


In [5]:
def price_storage_contract(
        injection_dates,
        withdrawal_dates,
        volume_per_injection,
        volume_per_withdrawal,
        max_storage,
        injection_fee=0,        # cost per injection
        withdrawal_fee=0,       # cost per withdrawal
        monthly_storage_cost=0  # monthly fee
    ):

    current_storage = 0
    total_buy_cost = 0
    total_sell_revenue = 0
    total_fees = 0

    # 1) Process injections
    for date in injection_dates:
        price = get_price(date)

        if current_storage + volume_per_injection > max_storage:
            raise ValueError("Storage limit exceeded")

        current_storage += volume_per_injection
        total_buy_cost += price * volume_per_injection
        total_fees += injection_fee

    # 2) Process withdrawals
    for date in withdrawal_dates:
        price = get_price(date)

        if current_storage - volume_per_withdrawal < 0:
            raise ValueError("Cannot withdraw more than stored")

        current_storage -= volume_per_withdrawal
        total_sell_revenue += price * volume_per_withdrawal
        total_fees += withdrawal_fee

    # 3) Storage cost for the entire period
    start = pd.to_datetime(injection_dates[0])
    end = pd.to_datetime(withdrawal_dates[-1])
    total_months = (end.year - start.year) * 12 + (end.month - start.month)
    total_storage_cost = total_months * monthly_storage_cost

    # Contract value
    value = total_sell_revenue - total_buy_cost - total_fees - total_storage_cost
    return value


#Test the Pricing Model

In [6]:
test_value = price_storage_contract(
    injection_dates = ["2023-06-01", "2023-07-01"],
    withdrawal_dates = ["2023-12-01", "2024-01-01"],
    volume_per_injection = 50000,
    volume_per_withdrawal = 50000,
    max_storage = 200000,
    injection_fee = 10000,
    withdrawal_fee = 10000,
    monthly_storage_cost = 50000
)

test_value


5326.70184743579